# Run All

## 1. Import Lib

Thêm các thư viện

In [1]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)
import torch
import torch.nn as nn
import numpy as np
import time
import os
import matplotlib.pyplot as plt
import torchvision.utils as vutils
from torchvision.utils import save_image

True
12.4


Kiểm tra 

In [2]:
import torch

if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Có {num_gpus} GPU(s) được phát hiện:")
    
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("Không có GPU nào được phát hiện.")


Có 1 GPU(s) được phát hiện:
GPU 0: Quadro RTX 4000 with Max-Q Design


In [3]:
# Kiểm tra nếu có GPU khả dụng, nếu không thì sử dụng CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

## Get Image Preprocessing

In [4]:
import os
from PIL import Image
import torch
from torchvision import transforms

def load_and_shuffle_images(folder_path, mean, std, img_shape=(128, 128)):
    images = []
    
    # Định nghĩa các transform để xử lý ảnh
    transform = transforms.Compose([
        transforms.Resize(img_shape),  # Resize ảnh về kích thước (128, 128)
        transforms.ToTensor(),  # Chuyển đổi ảnh thành tensor và chuẩn hóa về [0, 1]
        transforms.Normalize(mean, std)  # Chuẩn hóa về [-1, 1]
    ])
    
    # Đọc và transform ảnh
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = Image.open(img_path).convert("RGB")  # Đảm bảo ảnh có 3 kênh (RGB)
        img_tensor = transform(img)  # Áp dụng các transform lên ảnh
        images.append(img_tensor)
    
    # Shuffle các tensor ảnh
    images_tensor = torch.stack(images)  # Chuyển danh sách ảnh thành tensor với kích thước (N, 3, 128, 128)
    shuffled_indices = torch.randperm(images_tensor.size(0))  # Tạo thứ tự ngẫu nhiên để shuffle
    shuffled_images_tensor = images_tensor[shuffled_indices]  # Trộn các tensor ảnh
    
    return shuffled_images_tensor

# Thiết lập mean và std để chuẩn hóa ảnh
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

# Thay đổi đường dẫn tới thư mục chứa ảnh 
folder_path = rf'D:\Download\Edge\stinkbug\Merge\Labeling\Stink_bugs\Categories\Images\Inputs\128\adult'  
img_shape = (128, 128)

# Load và shuffle ảnh
real_images = load_and_shuffle_images(folder_path, mean, std, img_shape)

# Kiểm tra kích thước tensor
print(f"Loaded and shuffled {real_images.size(0)} images with shape {real_images.shape}")


Loaded and shuffled 806 images with shape torch.Size([806, 3, 128, 128])


In [5]:
import torch
import torch.nn as nn

# Generator
class Generator(nn.Module):
    def __init__(self, noise_dim, channels):
        super(Generator, self).__init__()
        
        self.model = nn.Sequential(
            # Dense layer equivalent
            nn.Linear(noise_dim, 8 * 8 * 1024),
            nn.LeakyReLU(0.2),
            
            # Reshape to (batch_size, 1024, 8, 8)
            nn.Unflatten(1, (1024, 8, 8)),
            
            # First Conv2DTranspose: (8x8) -> (16x16)
            nn.ConvTranspose2d(1024, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),
            
            # Second Conv2DTranspose: (16x16) -> (32x32)
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            
            # Third Conv2DTranspose: (32x32) -> (64x64)
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            
            # Fourth Conv2DTranspose: (64x64) -> (128x128)
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            
            # Output layer: Conv2D
            nn.Conv2d(64, channels, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)




In [6]:
# Discriminator
class Discriminator(nn.Module):
    def __init__(self, channels, img_size):
        super(Discriminator, self).__init__()
        
        self.model = nn.Sequential(
            # First Conv2D layer: (128x128) -> (64x64)
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Second Conv2D layer: (64x64) -> (32x32)
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Third Conv2D layer: (32x32) -> (16x16)
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Fourth Conv2D layer: (16x16) -> (8x8)
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Flatten and Dense output
            nn.Flatten(),
            nn.Dropout(0.4),
            nn.Linear(512 * (img_size // 16) * (img_size // 16), 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [7]:
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

# Thiết lập các siêu tham số
NOISE_DIM = 100
BATCH_SIZE = 16
STEPS_PER_EPOCH = real_images.size(0) // BATCH_SIZE
EPOCHS = 1000
img_size = 128
channels = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Khởi tạo Generator và Discriminator và chuyển chúng sang GPU (nếu có)
generator = Generator(NOISE_DIM, channels).to(device)
discriminator = Discriminator(channels, img_size).to(device)

# Khởi tạo Optimizer
optimizer_g = optim.Adam(generator.parameters(), lr=0.0001, betas=(0.5, 0.999))
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.000002, betas=(0.5, 0.999))

# Hàm mất mát
criterion = nn.BCELoss()

# Biến lưu trữ lịch sử loss
G_losses = []
D_losses = []

save_image_dir = rf'D:\Download\Edge\stinkbug\Merge\Labeling\Stink_bugs\Code_split_data\DC_Gan\generated_images\v8'
model_dir = rf"D:\Download\Edge\stinkbug\Merge\Labeling\Stink_bugs\Code_split_data\DC_Gan\models\v8"
filename = rf'D:\Download\Edge\stinkbug\Merge\Labeling\Stink_bugs\Code_split_data\DC_Gan\json\training_log.json'
image_loss_path = rf"D:\Download\Edge\stinkbug\Merge\Labeling\Stink_bugs\Code_split_data\DC_Gan\models\v8"

In [9]:
pip freeze > requirements.txt  # Python3

Note: you may need to restart the kernel to use updated packages.


In [12]:
import platform
print(platform.python_version())


3.10.0
